# Install Libraries

In [1]:
!pip install opencv-python
!pip install imutils
!pip install tensorflow

# Import Libraries

In [2]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

import os 

from fl_utils import *
from functions import *

# Data Set up

## Path to Data & Loading Data

### Read the View Text files

In [3]:

def readviewtextfile (filename):
    with open(filename) as f:
        lines = f.readlines()
    f.close()

    num_par = len (lines)
    num_feat = len (lines[0].split (","))
    feat_matrix = np.zeros ((num_par,num_feat))

    for i in range (num_par):
        clean_items = lines[i].replace ('\n','')
        seperated_items = clean_items.split(",")

        for j in range (num_feat):
            feat_matrix [i][j] = float (seperated_items[j])
    return feat_matrix


In [4]:
avg_view = readviewtextfile ('avg_view.txt')
loc_view = readviewtextfile ('loc_view.txt')
# trend view
phq_score = readviewtextfile ('phq_score.txt')

### Adjustment of the Labels

In [5]:
label_encoder = preprocessing.LabelEncoder()
 
label_list = label_encoder.fit_transform(phq_severity_sri (phq_score))

## TESTING PURPOSES BY COMBINING ALL VIEWS

In [6]:
ft_matrix = np.hstack ((avg_view,loc_view))

## Test-Train Split

In [136]:
#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(ft_matrix, 
                                                    label_list, 
                                                    test_size=0.2, 
                                                    random_state=52)

y_train = y_train.reshape (len(y_train),1)
y_test = y_test.reshape (len(y_test),1)

# Federated Learning Set-up

## Create Clients

In [138]:
#create clients
clients = create_clients(X_train, y_train, num_clients=1, initial='client')

In [139]:
print (clients)

{'client_1': [(array([1.43128143e+06, 6.47583740e+04, 1.06596562e+04, 8.73559000e+05,
       1.66200000e+03, 6.83662000e+05, 6.70000000e+01, 3.70941103e+06,
       1.58366585e+06, 1.22394631e+06, 2.29210400e+06, 2.79000000e+02,
       9.11926571e-02, 4.15755767e-01, 0.00000000e+00, 0.00000000e+00,
       3.64891080e-01, 0.00000000e+00, 9.65790146e-01, 5.79525147e-02,
       1.77738281e+02]), array([0], dtype=int64)), (array([1.00861752e+06, 7.66702799e+04, 2.25077487e+04, 1.60459600e+06,
       2.11100000e+03, 1.76565000e+06, 1.84000000e+02, 2.54169057e+06,
       2.34473751e+06, 1.48653887e+06, 2.13746400e+06, 2.26000000e+02,
       1.95256628e+02, 4.54979900e-01, 0.00000000e+00, 0.00000000e+00,
       3.58297598e-01, 0.00000000e+00, 9.42075996e-01, 2.31464319e-01,
       1.18090688e+03]), array([0], dtype=int64)), (array([8.98827061e+05, 3.63385969e+04, 1.30567105e+04, 8.08665000e+05,
       1.37300000e+03, 2.07537900e+06, 1.52000000e+02, 2.37942663e+06,
       9.97025744e+05, 8.8271

## Batch Cliets

In [140]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data,10)
    


In [141]:
clients_batched['client_1']

<BatchDataset shapes: ((None, 21), (None, 1)), types: (tf.float64, tf.int64)>

In [142]:
for element in clients_batched['client_1']:
    print (element)

(<tf.Tensor: shape=(10, 21), dtype=float64, numpy=
array([[1.18702173e+06, 4.82197639e+04, 2.61792601e+04, 8.01298000e+05,
        1.91400000e+03, 2.67937100e+06, 2.44000000e+02, 3.28203986e+06,
        1.04349404e+06, 8.09031053e+05, 3.22506600e+06, 3.33000000e+02,
        7.63562117e+01, 5.70593693e-01, 0.00000000e+00, 0.00000000e+00,
        3.20147507e-01, 0.00000000e+00, 9.61268904e-01, 1.69125784e-01,
        1.05337791e+03],
       [9.96711293e+05, 8.28447753e+04, 3.93374658e+04, 1.50598600e+06,
        2.73700000e+03, 3.17223900e+06, 2.68000000e+02, 2.27332069e+06,
        1.86439128e+06, 2.00419201e+06, 3.49366900e+06, 2.41000000e+02,
        2.68260972e-02, 4.37547344e-01, 3.63411416e-04, 6.05685693e-04,
        3.69029492e-01, 3.35905120e-01, 8.95027624e-01, 4.06629834e-01,
        1.99593622e+02],
       [1.22433786e+06, 6.75516174e+04, 2.39417979e+04, 8.76731000e+05,
        1.93600000e+03, 2.26759100e+06, 2.08000000e+02, 3.33224197e+06,
        1.14183841e+06, 9.13127737e

In [143]:
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [144]:
for element in test_batched:
    print (element)

(<tf.Tensor: shape=(8, 21), dtype=float64, numpy=
array([[1.05801391e+06, 7.28335484e+04, 1.12204081e+04, 1.63534700e+06,
        1.95500000e+03, 2.21243100e+06, 1.74000000e+02, 2.82949724e+06,
        2.02798494e+06, 1.40344378e+06, 2.91307000e+06, 2.28000000e+02,
        4.04376639e-02, 5.22095791e-01, 0.00000000e+00, 0.00000000e+00,
        3.39312248e-01, 0.00000000e+00, 9.86387435e-01, 3.77225131e-01,
        7.76116026e+01],
       [1.26311809e+06, 3.97224432e+04, 1.13860736e+04, 1.15428400e+06,
        2.26000000e+03, 2.40723400e+06, 2.17000000e+02, 3.22831497e+06,
        1.60486187e+06, 1.05054258e+06, 2.99284800e+06, 3.01000000e+02,
        4.53994089e-06, 3.09391634e-01, 1.39035775e-01, 2.31726292e-02,
        7.24522838e-01, 6.59489108e-01, 5.93448508e-01, 2.26834193e-01,
        6.74015530e+01],
       [9.35714745e+05, 3.01746823e+04, 3.31884205e+03, 3.61188000e+05,
        9.02000000e+02, 5.98908000e+05, 4.60000000e+01, 1.86411052e+06,
        4.91308218e+05, 5.54492256e+

## 3 Layer MLP (multi-layer perception) model

In [145]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

## Declearing model optimizer, loss function and a metric

In [146]:
#Learning Rate
lr = 0.01 

# number global epochs (aggregations)
comms_round = 10

#Loss Function
loss='categorical_crossentropy'

#Evaluation Metric
metrics = ['accuracy']

#Model optimization using SGD (Stochastic Gradient Descent)
optimizer = SGD(learning_rate=lr, decay=lr / comms_round, momentum=0.9)              

# Model Aggregation (Federated Averaging) and Testing 

In [147]:
shape = 21 # Num of Feats
classes = 1 # Time series therefore is 1

In [150]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(shape, classes)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(shape, classes)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        print(Y_test)
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 0 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 1 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 2 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 3 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 4 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 5 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]], shape=(8, 1), dtype=int64)
comm_round: 6 | global_acc: 100.000% | global_loss: nan
tf.Tensor(
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [

# SGD Comparison to Federated Learning Accuracy

In [ ]:
#Creating SGD Dataset with all hyperparameters equal to that of FL, however with a batch size of 320 as there are not multiple independent models or multiple clients to have said models
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320) 

#Initializing Model
smlp_SGD = SimpleMLP()

#Generating model with a shape of 784, and 10 classes
SGD_model = smlp_SGD.build(shape, classes) 

#Compilation of Model using model paramaters
SGD_model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)